In [ ]:
!pip install langchain_core langgraph langchain-cerebras

In [2]:
import os, getpass

In [3]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [14]:
_set_env('CEREBRAS_API_KEY')

In [79]:
# LLM
# from langchain_anthropic import ChatAnthropic
# llm = ChatAnthropic(model_name='claude-3-5-sonnet-latest')


from langchain_core.prompts import ChatPromptTemplate
from langchain_cerebras import ChatCerebras

llm = ChatCerebras(
    model="llama-3.3-70b",
)

In [80]:
# Schema for structured output
from pydantic import BaseModel, Field
class SearchQuery(BaseModel):
    # The below arguments modify the response.
    # Essentially, .invoke() takes a prompt.
    # Based on the prompt, search_query is generated based on description
    # justification is generated again based on justification
    # response is generated again based on description
    # So, the actual prompt acts as main context
    # The descriptions are additional prompts.
    # So, the llm generates a response for each argument based on the description.
    # The arguments in the field part say: hey llm, take the prompt. give me a search_query such that the search_query matches the description,
    # a justification of the search_query, why the search_query you have generated is relevant to the user's request,
    # response, results for the search query
    # Essentially, each response to the argument acts as context to the llm for the response to the next argument.
    search_query: str = Field(None, description='Query that is optimized web search.')
    search_query_validation: str = Field(None, description='What is the search_query you have generated?')
    justification: str = Field(None, justification="Why this query is relevant to the user's request.")
    response: str = Field(None, description='Give results for the search_query')
    # sadfasf: str = Field(None, description='What is the role of arguments in SearchQuery(BaseModel)?')

In [81]:
# Augment the LLM with schema for structured output
structured_llm = llm.with_structured_output(SearchQuery)

In [82]:
# Invoke the augmented LLM
output = structured_llm.invoke('How does Calcium CT score relate to high cholestrol?')
print(output.search_query)
print(output.justification)

Calcium CT score and high cholesterol
This query is relevant to the user's request because it directly relates to the relationship between Calcium CT score and high cholesterol.


In [83]:
output.response

'The search results will provide information on how Calcium CT score relates to high cholesterol.'

In [84]:
output.search_query_validation

'Calcium CT score and high cholesterol'

In [ ]:
# Define a tool
def multiply(a, b):
    return a * b

# Augment the LLM with tools
llm_on_tools = llm.bind_tools([multiply])

# Invoke the LLM with input that triggers the tool call
msg = llm_on_tools.invoke('What is 2.5 * 3.5')

# Get the tool call
msg.tool_calls[0]['args']


[{'name': 'multiply',
  'args': {'a': 2.5, 'b': 3.5},
  'id': '2fadc2ad1',
  'type': 'tool_call'}]

In [96]:
a = msg.tool_calls[0]['args']['a']
b = msg.tool_calls[0]['args']['b']

c = multiply(a,b)
print(llm.invoke(f'{c}').content)

That's a numerical value: 8.75. 

Would you like to perform some calculations or operations with it, or is there something else I can help you with?


In [97]:
from typing_extensions import TypedDict

# Graph state
class State(TypedDict):
    topic: str
    joke: str
    improved_joke: str
    final_joke: str

In [ ]:
# Nodes
def generate_joke(state: State):
    

In [125]:
from langchain_cerebras import ChatCerebras
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, MessagesState

# Build graph
graph = StateGraph(MessagesState)
graph.add_node("chat", lambda state: {
    "messages": state["messages"] + [
        AIMessage(content=str(llm.invoke(state["messages"][-1].content)))
    ]
})
graph.set_entry_point("chat")
chain = graph.compile()

# Run
msg = HumanMessage(content=input("Your message: "))
response = chain.invoke({"messages": [msg]})
import re
# content = re.search(r"content='([^']*)'", response["messages"][-1].content).group(1)

parts = response["messages"][-1].content.split(' additional_kwargs=')
content = parts[0].split("=", 1)[1].strip("'")
print(content)

"A Large Language Model (LLM) is a type of artificial intelligence (AI) designed to process and understand human language. It's a computer program that uses complex algorithms and large amounts of data to generate human-like text, answer questions, and even converse with humans.\n\nLLMs are trained on vast amounts of text data, which allows them to learn patterns, relationships, and structures of language. This training enables them to:\n\n1. **Generate text**: Create coherent and often grammatically correct text based on a prompt or input.\n2. **Answer questions**: Respond to questions by retrieving information from their vast knowledge base.\n3. **Translate languages**: Translate text from one language to another.\n4. **Summarize content**: Condense long pieces of text into shorter summaries.\n5. **Converse**: Engage in natural-sounding conversations, using context and understanding to respond to questions and statements.\n\nSome key characteristics of LLMs include:\n\n* **Scalabilit

In [4]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_cerebras import ChatCerebras

# Use Cerebras LLM
llm = ChatCerebras(model="llama-3.3-70b")

class State(TypedDict):
    topic: str
    joke: str
    improved_joke: str
    final_joke: str

def generate_joke(state: State):
    msg = llm.invoke(f"Write a short joke about {state['topic']}")
    return {"joke": msg.content}

def improve_joke(state: State):
    msg = llm.invoke(f"Make this joke funnier with wordplay: {state['joke']}")
    return {"improved_joke": msg.content}

def polish_joke(state: State):
    msg = llm.invoke(f"Add a surprising twist to this joke: {state['improved_joke']}")
    return {"final_joke": msg.content}

# def check_punchline(state: State):
#     return "Pass" if "?" in state["joke"] or "!" in state["joke"] else "Fail"

def check_punchline(state: State):
    prompt = f"Does this joke have a punchline? Answer only yes or no. Joke: {state['joke']}"
    response = llm.invoke(prompt)
    return "Pass" if "yes" in response.content.lower() else "Fail"

graph = StateGraph(State)
graph.add_node("generate", generate_joke)
graph.add_node("improve", improve_joke)
graph.add_node("polish", polish_joke)
graph.set_entry_point("generate")
graph.add_conditional_edges("generate", check_punchline, {"Pass": "improve", "Fail": END})
graph.add_edge("improve", "polish")
graph.add_edge("polish", END)
app = graph.compile()

# To run (requires CEREBRAS_API_KEY environment variable):
from IPython.display import display, Markdown

result = app.invoke({"topic": "computers"})
for key, value in result.items():
    display(Markdown(f"### {key.replace('_', ' ').title()}\n\n{value}"))

### Topic

computers

### Joke

Why did the computer go to the doctor?

It had a virus! (get it?)

### Improved Joke

A classic one. Here are a few variations with some added wordplay:

1. Why did the computer go to the doctor? It had a virus, and it was feeling a little "dis-connected"!
2. Why did the computer go to the doctor? It had a virus, and it was in a bit of a "re-boot" situation!
3. Why did the computer go to the doctor? It had a virus, and it was struggling to "byte" back the infection!
4. Why did the computer go to the doctor? It had a virus, and it was experiencing some "glitchy" symptoms!
5. Why did the computer go to the doctor? It had a virus, and it needed to get "patched" up!

Hope these variations "compute" with you!

### Final Joke

Here are the variations with a surprising twist added to each:

1. Why did the computer go to the doctor? It had a virus, and it was feeling a little "dis-connected"! But little did the doctor know, the virus was actually a government-created AI designed to infiltrate the medical system, and the computer was just a pawn in a much larger game of cyber espionage.
2. Why did the computer go to the doctor? It had a virus, and it was in a bit of a "re-boot" situation! However, when the doctor tried to reboot the system, the computer suddenly sprang to life and began reciting the entire script of "The Matrix," revealing that it had become self-aware and was now questioning the nature of its digital reality.
3. Why did the computer go to the doctor? It had a virus, and it was struggling to "byte" back the infection! But as the doctor examined the computer's code, she discovered that the virus was actually a message from an alien civilization, hidden in a series of 1s and 0s, and the computer was unwittingly serving as an intergalactic ambassador.
4. Why did the computer go to the doctor? It had a virus, and it was experiencing some "glitchy" symptoms! Unbeknownst to the doctor, the virus was a side effect of a secret experiment to merge human and artificial intelligence, and the computer was now exhibiting bizarre, almost-human behavior, including a penchant for 80s pop music and a fear of spiders.
5. Why did the computer go to the doctor? It had a virus, and it needed to get "patched" up! But as the doctor applied the patch, the computer's screen flickered to life, revealing a hidden virtual world where the computer had been living a secret life as a digital rockstar, and the virus was just a publicity stunt to promote its new album, "System Failure."

In [119]:
print(response["messages"][-1].content)

content='Hello. How can I help you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'llama-3.3-70b', 'system_fingerprint': 'fp_7c596c6482a5e3eac4a6', 'id': 'chatcmpl-b1f74f1c-f19c-4c50-95f4-538322146ee7', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--4586bc09-55e3-4b49-bf03-4f5ba73f8187-0' usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


In [120]:
import re
content = re.search(r"content='([^']*)'", response["messages"][-1].content).group(1)
print(content)

Hello. How can I help you today?
